In [1]:
import pandas as pd
from datetime import datetime

data = {
    'Fecha': [
        '2025-01-10 05:03:27', '2025-01-08 07:05:51', '2025-01-11 14:05:51',
        '2024-12-17 18:47:51', '2024-12-27 22:50:51', '2025-01-15 09:39:51',
        '2025-01-07 19:14:51', '2025-01-21 18:40:51', '2024-12-23 15:08:51',
        '2024-12-28 22:38:51', '2024-12-11 17:35:51', '2024-12-21 11:04:51',
        '2024-12-21 06:08:51', '2025-01-22 01:35:51', '2025-01-11 05:04:51'
    ],
    'Id': [
        24942603, 95822412, 95822412, 24942603, 95822412, 24942603,
        24942603, 13356886, 24942603, 95822412, 13356886, 24942603,
        13356886, 95822412, 24942603
    ],
    'MontoRecibido': [
        85.46, 3181.93, 332.89, 448.45, 212.74, 136.53,
        81.3, 347.57, 1397.88, 493.35, 434.89, 439.92,
        221.06, 254.03, 425.35
    ]
}

df = pd.DataFrame(data)
df['Fecha'] = pd.to_datetime(df['Fecha'])

hoy = pd.to_datetime(datetime.now().date()) 
print(hoy)
df['Dias_Hasta_Hoy'] = (hoy - df['Fecha'].dt.normalize()).dt.days
df = df.sort_values(by=['Id', 'Fecha'], ascending=True).reset_index(drop=True)
df

2025-01-23 00:00:00


,Fecha,Id,MontoRecibido,Dias_Hasta_Hoy
0,2024-12-11 17:35:51,13356886,434.89,43
1,2024-12-21 06:08:51,13356886,221.06,33
2,2025-01-21 18:40:51,13356886,347.57,2
3,2024-12-17 18:47:51,24942603,448.45,37
4,2024-12-21 11:04:51,24942603,439.92,33
5,2024-12-23 15:08:51,24942603,1397.88,31
6,2025-01-07 19:14:51,24942603,81.30,16
7,2025-01-10 05:03:27,24942603,85.46,13
8,2025-01-11 05:04:51,24942603,425.35,12
9,2025-01-15 09:39:51,24942603,136.53,8


In [2]:
df_proc = df.groupby(by=['Id']).apply(
        lambda group: pd.Series({
            'CantTransfRecib': len(group),

            'MtoProm_u30d': round((group[(group['Dias_Hasta_Hoy'] < 30)]['MontoRecibido']).mean(), 1),
            'MtoProm_u60d': round((group[(group['Dias_Hasta_Hoy'] < 60)]['MontoRecibido']).mean(), 1),

            'CantTrfRec_u30d': (group[(group['Dias_Hasta_Hoy'] < 30)]['MontoRecibido']).count(),
            'CantTrfRec_u60d': (group[(group['Dias_Hasta_Hoy'] < 60)]['MontoRecibido']).count(),

            'MtoTot_u30d': round((group[(group['Dias_Hasta_Hoy'] < 30)]['MontoRecibido']).sum(), 1),
            'MtoTot_u60d': round((group[(group['Dias_Hasta_Hoy'] < 60)]['MontoRecibido']).sum(), 1),

            'MtoTot_10_30d': round((group[(group['Dias_Hasta_Hoy'] >= 10) &
                                          (group['Dias_Hasta_Hoy'] < 30)]['MontoRecibido']).sum(), 1),

            'UltMaxRec_Mayor1000': group.loc[group['MontoRecibido'].idxmax(), 'Fecha']
                if max(group['MontoRecibido']) > 1000
                else None,
            'DiasMaxRec_Mayor1000': group.loc[group['MontoRecibido'].idxmax(), 'Dias_Hasta_Hoy']
                if max(group['MontoRecibido']) > 1000
                else None,
        }), include_groups=False
    ).reset_index()

df_proc['UltMaxRec_Mayor1000'] = df_proc['UltMaxRec_Mayor1000'].dt.date
df_proc

,Id,CantTransfRecib,MtoProm_u30d,MtoProm_u60d,CantTrfRec_u30d,CantTrfRec_u60d,MtoTot_u30d,MtoTot_u60d,MtoTot_10_30d,UltMaxRec_Mayor1000,DiasMaxRec_Mayor1000
0,13356886,3.0,347.6,334.5,1.0,3.0,347.6,1003.5,0.0,NaT,NaN
1,24942603,7.0,182.2,430.7,4.0,7.0,728.6,3014.9,592.1,2024-12-23,31.0
2,95822412,5.0,895.0,895.0,5.0,5.0,4474.9,4474.9,4220.9,2025-01-08,15.0
